# Plotting and analysing models

In [ ]:
# Start with imports
# These should always live at the top of a notebook

# Import commonly used external libraries - you won't always need these, but most of the time you will
import numpy as np
import pandas as pd

# Import our project interface - this is the main method of accessing our models
from autumn.tools.project import get_project

# Also include our custom pretty printer - it can make things a lot easier to read!
from autumn.tools.utils.display import pretty_print

In [ ]:
model_name = 'covid_19'
model_region = 'malaysia'

project = get_project(model_name, model_region)

## Working with parameters

AuTuMN provides a number of facilities for interacting with model parameters, but the basic pattern is always the same;  
1. Get some parameters
2. Modify them (or not)
3. Build and run a model

In [ ]:
# Run the model with unmodified baseline parameters
# This command returns a summer CompartmentalModel object that contains the completed run data

params = project.param_set.baseline

m = project.run_baseline_model(params)

In [ ]:
# Summer provides convenience functions to access model outputs (and derived outputs) as pandas DataFrames
# This is the recommended way of using model outputs in an interactive context

outputs_df = m.get_outputs_df()
doutputs_df = m.get_derived_outputs_df()

In [ ]:
# Let's have a quick look at one of the derived outputs - pandas makes it easy to plot directly

doutputs_df['accum_deaths'].plot()

### Changing parameters

In the above example, we simply ran the model with the 'default' parameters (ie those specified in the project file)  
Here we will modify parameters programatically and compare outputs from different runs

In [ ]:
# Have a look at our baseline params; using pretty printing can make it easier to see what's going on

params_baseline = project.param_set.baseline

pretty_print(params_baseline)

In [ ]:
# Let's say we are interested in experimenting with a single parameter - contact rate
# We'll take a look at the existing value first

params_baseline['contact_rate']

In [ ]:
# To change a parameter value, the AuTuMN parameters API creates a non-destructive copy of the parameters with updates applied
# This is the recommended way of interacting with parameters, since it includes 
# automatic validation facilities and other niceties
# 
# In performance-sensitive contexts, you may want to interact with parameter dictionaries directly

# Updates are passed in via dictionaries
updates = {'contact_rate': 0.03}

# Create a new parameter set - params_baseline remains unchanged
new_params = params_baseline.update(updates)

new_params['contact_rate']

In [ ]:
# Run a model with the new parameters, and get its derived outputs
new_do_df = project.run_baseline_model(new_params).get_derived_outputs_df()

In [ ]:
# Examine the outputs
# We'll plot the original and new outputs overlayed so we can compare them directly

doutputs_df['accum_deaths'].plot(label='orig')
ax = new_do_df['accum_deaths'].plot(label='new')
ax.legend()

In [ ]:
# Here we generate a range of values programatically and compare all of them
# This is not necessarily the best or fastest way to do this, but does demonstrate how easy it is to build
# tools for exploring model dynamics

def run_model_with_param_updates(baseline, update_dict):
    params = baseline.update(update_dict)
    m = project.run_baseline_model(params)
    return m

cr_comp_df = pd.DataFrame()

for contact_rate in np.linspace(0.01, 0.05, 5):
    cur_results = run_model_with_param_updates(params_baseline, {'contact_rate': contact_rate})
    cr_comp_df[contact_rate] = cur_results.get_derived_outputs_df()['accum_deaths']


In [ ]:
# Let's have a look at the outputs

ax = cr_comp_df.plot(title="Accum deaths by contact rate")
# bbox_to_anchor lets you place a plot legend specifically
# For plots with a lot of data, you may want to move it outside 
# of the plotting frame altogether as we have done here
ax.legend(bbox_to_anchor=(1.0,1.0))